In [1]:
%reload_ext autoreload
%autoreload 2

csqa_directory="/home/ubuntu/Desktop/CSQA_v9/train" 
wikidata_directory="/home/ubuntu/Desktop/wikidata"

import sys
import os
import time
import argparse
from collections import defaultdict
sys.path.insert(0, os.path.dirname(os.path.dirname(os.path.abspath(os.path.join(os.path.abspath(''), os.pardir)))))

from allennlp.common.testing.test_case import AllenNlpTestCase
from allennlp.common import Params
from allennlp.data.dataset_readers.semantic_parsing.csqa.csqa import CSQADatasetReader
from allennlp.state_machines.states.grammar_statelet import GrammarStatelet
from allennlp.state_machines.states.grammar_based_search_state import GrammarBasedSearchState
from allennlp.semparse.domain_languages.csqa_language import CSQALanguage
from allennlp.semparse.domain_languages.csqa_language import Entity, Predicate
from allennlp.semparse.domain_languages import START_SYMBOL



Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:

params = {'lazy': False,
          # 'kg_path':  f'{AllenNlpTestCase.FIXTURES_ROOT}/data/csqa/sample_kg.p',
          'kg_path':  f'{wikidata_directory}/wikidata_short_1_2_full.p',
          # 'kg_type_datapath':  f'{wikidata_directory}/child_all_parents_till_5_levels_full.p',
          'kg_type_data_path':  f'{wikidata_directory}/par_child_dict_full.p',
          'entity_id2string_path':  f'{AllenNlpTestCase.FIXTURES_ROOT}/data/csqa/sample_entity_id2string.json',
          'predicate_id2string_path': f'{AllenNlpTestCase.FIXTURES_ROOT}/data/csqa/filtered_property_wikidata4.json'
         }


reader = CSQADatasetReader.from_params(Params(params))
qa_path = f'{AllenNlpTestCase.FIXTURES_ROOT}/data/csqa/sample_qa.json'
dataset = reader.read(qa_path)
# print_instance_info = True
print_instance_info = False
logical_forms_found = 0

0it [00:00, ?it/s]

Loading wikidata graph
Loading wikidata type graph


Your label namespace was 'denotations'. We recommend you use a namespace ending with 'labels' or 'tags', so we don't add UNK and PAD tokens by default to your vocabulary.  See documentation for `non_padded_namespaces` parameter in Vocabulary.
16it [01:23,  5.21s/it]


In [8]:
#language.kg_context.entity_id2string['Q18636161']
language.kg_data[1148698]

{735: [2827747, 2827747],
 725: [109430, 266670, 109430],
 1441: [185143],
 170: [313945, 313945],
 21: [6581072, 6581072],
 31: [15632617],
 -674: [185143]}

In [9]:
print_debug_info = True

logical_forms = ["(find (get Q12122755) P495)",
                 "(union (find (get Q18577469) P710) (find (get Q12122755) P161))",
                 "(count (find (get Q18643299) P551))",
                 "(less (find (get Q6619679) P-1) P725 (count (find (get Q11609133) P725)))",
                 "(intersection (find (get Q18636161) P-710) (find (get Q18638257) P-710))",
                 "(intersection (find (get Q6581097) P-21) (find (get Q4608649) P-1344))",
                 "(most (find (get Q95074) P-1) P725 3)"
                 ]

# for instance in dataset[0:5]:

for i, instance in enumerate(dataset[0:10]):
    instance = dataset[i]
    instance_start_time = time.time()

    question: str = instance['question'].tokens
    language: CSQALanguage = instance['world'].metadata
    expected_result = instance['expected_result'].metadata
    predicates = instance['question_predicates'].metadata


    if i != 6Untitled:
        continue

    print("question: {}".format(question))
    print("question entities {}".format(language._question_entities))

    # logical_form = logical_forms[i]
    # logical_form = "(find (get Q6619679) P-1)"
    # logical_form = "(count (find (get Q11609133) P725))"

    # logical_form = "(larger (find (get Q502895) P-1) P1344 0)"
    # logical_form = "(find (get Q502895) P-1)"
    # logical_form = "(find (get Q4608649) P-1344)"
    logical_form = logical_forms[i]
    
#     logical_form = "(most (find (get Q95074) P-1) P725 3)"
    
    # logical_form = "(equal (find (get Q95074) P-1) P725 0)"
    # logical_form = "(find (get Q95074) P-1)"
    # logical_form = "(intersection (find (get Q6581097) P-21) (find (get Q4608649) P-1344))"
    query_result = language.execute(logical_form)
    query_result = set(query_result) if isinstance(query_result, list) else query_result
#     print(query_result)
    if print_debug_info:
        if isinstance(expected_result, (list,set)):
            #TODO check comp_wikidata_rev
        #     print(list(query_result)[:10])
            print("len query result {}".format(len(query_result)))
            print("len expected result {}".format(len(expected_result)))

            print(query_result.intersection(expected_result))
            print("len query intersect expected result {}".format(len(query_result.intersection(expected_result))))
            print("len expected res diff query result {}".format(len(expected_result.difference(query_result))))
            print("expected res diff query result {}".format(expected_result.difference(query_result)))
            print("len query result difference expected result {}".format(len(query_result.difference(expected_result))))
    print(query_result == expected_result)

question: [which, fictional, characters, had, their, voice, dubbing, done, by, atmost, 3, people, ?]
question entities []
len query result 29
len expected result 39
{Entity(name='Q1529229', id=1529229), Entity(name='Q21837905', id=21837905), Entity(name='Q21847055', id=21847055), Entity(name='Q247120', id=247120), Entity(name='Q2442950', id=2442950), Entity(name='Q7810', id=7810), Entity(name='Q2725247', id=2725247), Entity(name='Q21847215', id=21847215), Entity(name='Q21847105', id=21847105), Entity(name='Q21847089', id=21847089), Entity(name='Q551327', id=551327), Entity(name='Q21847103', id=21847103), Entity(name='Q21837832', id=21837832), Entity(name='Q2570349', id=2570349), Entity(name='Q605125', id=605125), Entity(name='Q21847086', id=21847086), Entity(name='Q21847115', id=21847115), Entity(name='Q3498590', id=3498590), Entity(name='Q28465', id=28465), Entity(name='Q15710873', id=15710873), Entity(name='Q1984009', id=1984009), Entity(name='Q1621656', id=1621656), Entity(name='Q58